In [ ]:
""""
Traffic Monitoring and Analysis System - TMAS

This notebook is the "monitoring" part of TMAS. It has been designed to recognise 4 different classes 
of vehicle. Each time a vehicle is detected inside the central region of the frame, 
an entry in a database is made. 

TMAS has been created by Alexander Tkaczyk-Harrison, for
Goldsmtihs, University of London, BSc Creative Computing

TMAS uses the YOLO (You Only Look Once) Object Detector, which can be found at:
https://pjreddie.com/darknet/yolo/

@article{yolov3,
  title={YOLOv3: An Incremental Improvement},
  author={Redmon, Joseph and Farhadi, Ali},
  journal = {arXiv},
  year={2018}
}

The work of Rajeev Ratan must be noted. This project would not have happened without 
his excellent course on Udemy (https://www.udemy.com/share/100FTkBUESdFtaQHw=/)

The work of GitHub user "thtrieu" should also be noted, for converting the original YOLO algorithm
to work in Python, with tensorflow: https://github.com/thtrieu/darkflow

"""

from darkflow.net.build import TFNet # YOLO object detector
import numpy as np
import tensorflow as tf
import cv2
import time
import datetime
import sqlite3

# create connection and cursor to sqlite database
conn = sqlite3.connect('demo.db')
c = conn.cursor()


# tensorflow config options
# prints out which devices your operations and tensors are assigned to (set to True)
config = tf.ConfigProto(log_device_placement = False)

# GPU growth not needed, 
# by default Tensorflow maps almost all GPU memory to this process
config.gpu_options.allow_growth = False


# more tensorflow config options
with tf.Session(config = config) as sess:
    options = {
        'model' : './cfg/tiny-yolo-4c.cfg', # load custom tiny-yolo 4 class model
        'load' : 31500,                     # load from checkpoint 31500
        'threshold' : 0.5,                  # threshold to return predictions
        'gpu' : 1.0                         # GPU ID 
    }
    
    # pass these options into the YOLO algorithm
    tfnet = TFNet(options) 

In [ ]:
# create 4 tables (only if they do not exist already), 
# each table represents each class (cars, vans, HGVs, bikes), 
# columns are: vehicle type, unix time, 24 hour time, the date, and a value
def create_table():
    c.execute('CREATE TABLE IF NOT EXISTS cars(vehicle TEXT, unix REAL, hours TEXT, date TEXT, value REAL)')
    c.execute('CREATE TABLE IF NOT EXISTS vans(vehicle TEXT, unix REAL, hours TEXT, date TEXT, value REAL)')
    c.execute('CREATE TABLE IF NOT EXISTS HGVs(vehicle TEXT, unix REAL, hours TEXT, date TEXT, value REAL)')
    c.execute('CREATE TABLE IF NOT EXISTS bikes(vehicle TEXT, unix REAL, hours TEXT, date TEXT, value REAL)')
    
# define 4 functions, from lines 13, to 42... one for each class of vehicle
# when any of the functions are ran, add one to the corresponding table with fields
# approriately filled with the vehicles class, and the times and dates of entry
def car(date, hours, value, unix):
    
    # set string to be entered into the first column of the table
    vehicle = 'Car'
    
    # line 17 is SQLite syntax, the "?,?,?,?,?" part is needed to fill the 5 different columns
    c.execute("INSERT INTO cars (vehicle, unix, hours, date, value) VALUES (?, ?, ?, ?, ?)",
              # line 18 gets the variables out of the parameters of the function to put into the tabel
             (vehicle, unix, hours, date, value))
    
    # commit to the database defined on line 38 of previous cell
    conn.commit()

def van(date, hours, value, unix):
    vehicle = 'Van'
    c.execute("INSERT INTO vans (vehicle, unix, hours, date, value) VALUES (?, ?, ?, ?, ?)",
             (vehicle, unix, hours, date, value))
    conn.commit()
    
def HGV(date, hours, value, unix):
    vehicle = 'HGV'
    c.execute("INSERT INTO HGVs (vehicle, unix, hours, date, value) VALUES (?, ?, ?, ?, ?)",
             (vehicle, unix, hours, date, value))
    conn.commit()    

def bike(date, hours, value, unix):
    vehicle = 'Bike'
    c.execute("INSERT INTO bikes (vehicle, unix, hours, date, value) VALUES (?, ?, ?, ?, ?)",
             (vehicle, unix, hours, date, value))
    conn.commit()
    
    
# define check function with parameters: 
# frame width, dot location (central point of each result), size, and label
def check(fWidth, dLoc, size, label):
    # get unix time
    unix = time.time() 
    
    # create date from unix time
    date = str(datetime.datetime.fromtimestamp(unix).strftime('%Y-%m-%d')) 
    
    # create time from unix time
    hours = str(datetime.datetime.fromtimestamp(unix).strftime('%H:%M:%S')) 
    
    # value of entry
    value = 1
    
    # defines the central point of the screen
    center = fWidth//2
    
    # check if the location of the dot is inside this central area 
    # (size comes from line 6 of the cell below)
    if dLoc < center+size and dLoc > center-size:
        
        # checks the label assigned to each region of interest
        # depending on the label, run the corresponding function above
        # print confirmation of data going into the database
        if label == "car":                        
            car(date, hours, value, unix)         
            print("Adding to cars...")            
            
        elif label == "van":
            van(date, hours, value, unix)
            print("Adding to vans...")

        elif label == "HGV":
            HGV(date, hours, value, unix)
            print("Adding to HGVs...")

        elif label == "motor bike":
            bike(date, hours, value, unix)
            print("Adding to bikes...")

    return

# display function, this is where the magic happens!!
# parameters are: results, which come from YOLO,
# the image (webcam feed), width and height of the frame,
# the check function, and the size variable
def display(results, img, width, height, check, size):
    
    # draw lines around the central area (so you can see it on screen)
    cv2.line(img, ((width//2)+size, 0), ((height//2)+size, height),(0, 0, 0), 1)
    cv2.line(img, ((width//2)-size, 0), ((height//2)-size, height),(0, 0, 0), 1)

    # loops through each result, returned by the YOLO object detector
    # pulls the X and Y coordinate of the top left point, 
    # and stores them in appropriate variables
    # use the X and Y point of the bottom right location to
    # work out the width and height of each box
    # (look inside darkflow/net/flow.py to find the code which returns each result)
    for (i, result) in enumerate(results):
        x = result['topleft']['x']
        y = result['topleft']['y']
        w = result['bottomright']['x']-result['topleft']['x']
        h = result['bottomright']['y']-result['topleft']['y']
        
        # work out central point of each result
        w2 = (x+w//2)
        h2 = (y+h//2)
        
        # each result has a label assigned to it, these labels are what i named each class while
        # labelling my data set (car, van, HGV, and motor bike)
        # look at the "process_box" function, inside: darkflow/darkflow/net/yolo/predict.py 
        # to see how YOLO assigns the label. I believe its from the meta data which comes
        # from the XML files which i created when labelling the dataset.
        # lines 100 - 118 check the label returned by YOLO...
        # draw a rectangle around each result
        # put a small dot in the center of each result
        # run the check function (to see if the center of the box is in the center of the screen,
        # if it is, add one to the appropriate table)
        if result['label'] == str('car'):
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),1)
            cv2.circle(img, (w2, h2), 2, (0,255,0), 1)
            check(width, w2, size, str('car'))
            
        elif result['label'] == str('van'):
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),1)
            cv2.circle(img, (w2, h2), 2, (255,0,0), 1)
            check(width, w2, size, str('van'))
            
        elif result['label'] == str('HGV'):
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),1)
            cv2.circle(img, (w2, h2), 2, (0,0,255), 1)
            check(width, w2, size, str('HGV'))
            
        elif result['label'] == str('motor bike'):
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),1)
            cv2.circle(img, (w2, h2), 2, (255,255,0), 1)
            check(width, w2, size, str('motor bike'))
            
        # variable used to position the label of each result
        label_position = (x + int(w/2)), abs(y - 10)

        # lines 133 to 137 puts text above each result, add the label of the result, followed by a space, 
        # followed by the confidence of the prediction restricted to 4 charachters (for example: 0.76)
        # (each result could have over 10 decimal points, varible "d" is to keep it tidy)
        a = result['label']
        b = str('__')
        c = str(result['confidence'])
        d = c[0:4]
        cv2.putText(img, a + b + d, label_position , cv2.FONT_HERSHEY_SIMPLEX,0.5, (255,255,255), 1)
        
        
        # bear in mind that lines 101 - 150 are in a loop, running over every result
        
    return img

In [ ]:
# sets the width and height of the frame
F_W = 1024    
F_H = 1024

# size variable is passed throught he display function, into the check function
size = 12     

# lines 10 to 13 are OpenCV functions. In order they open the webcam feed, set the 
# width and height of the frame, and deactivate the autofocus
capture = cv2.VideoCapture(0)   
capture.set(3, F_W)
capture.set(4, F_H)
capture.set(cv2.CAP_PROP_AUTOFOCUS, 0)

# run create table function from line 4, of cell 2 (incase the tables do not exist)
create_table()

# infinately assign the webcam feed to the frame variable
while True: 
    ret, frame = capture.read()
    
    # ret obtains a return value from getting the camera frame, either true of false
    if ret:
        
        # webcam feed is passed into the YOLO algorithm, stored in the results variable
        results = tfnet.return_predict(frame)
        
        # results variable is passed into the display function, from line 76 of the above cell
        image = display(results, frame, F_W, F_H, check, size)
        
        # display the webcam feed in a new window, with boxes, labels, and confidence scores around each
        # result
        cv2.imshow('TMAS - Creative Project', image)
        
        # press enter to break the loop, and execute code on lines 41-44
        if cv2.waitKey(1) == 13:
            break
            
# close all connections (SQLite database connection, and OpenCV webcam connection)
c.close()
conn.close()
capture.release()
cv2.destroyAllWindows()

In [ ]:
# run this cell to close all connections (if the program crashes)
# then restart the program
c.close()
conn.close()
capture.release()
cv2.destroyAllWindows()